In [5]:
import csv
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [6]:
data_df = pd.read_csv('housing.csv') 

FileNotFoundError: [Errno 2] No such file or directory: 'housing.csv'

In [6]:
data_df.head(5)

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7049044568,https://reno.craigslist.org/apa/d/reno-beautif...,reno / tahoe,https://reno.craigslist.org,1148,apartment,1078,3,2.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/01616_daghmBUvTC...,Ridgeview by Vintage is where you will find al...,39.5483,-119.796,ca
1,7049047186,https://reno.craigslist.org/apa/d/reno-reduced...,reno / tahoe,https://reno.craigslist.org,1200,condo,1001,2,2.0,0,...,0,0,0,w/d hookups,carport,https://images.craigslist.org/00V0V_5va0MkgO9q...,Conveniently located in the middle town of Ren...,39.5026,-119.789,ca
2,7043634882,https://reno.craigslist.org/apa/d/sparks-state...,reno / tahoe,https://reno.craigslist.org,1813,apartment,1683,2,2.0,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00t0t_erYqC6LgB8...,2BD | 2BA | 1683SQFTDiscover exceptional servi...,39.6269,-119.708,ca
3,7049045324,https://reno.craigslist.org/apa/d/reno-1x1-fir...,reno / tahoe,https://reno.craigslist.org,1095,apartment,708,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00303_3HSJz75zlI...,MOVE IN SPECIAL FREE WASHER/DRYER WITH 6 OR 12...,39.4477,-119.771,ca
4,7049043759,https://reno.craigslist.org/apa/d/reno-no-long...,reno / tahoe,https://reno.craigslist.org,289,apartment,250,0,1.0,1,...,1,0,1,laundry on site,NaN,https://images.craigslist.org/01616_fALAWFV8zQ...,"Move In Today: Reno Low-Cost, Clean & Furnishe...",39.5357,-119.805,ca


In [7]:
data_df.type.value_counts()

apartment          318032
house               33266
townhouse           15885
condo                6238
duplex               5047
manufactured         4242
cottage/cabin         861
loft                  693
flat                  531
in-law                172
land                    8
assisted living         2
Name: type, dtype: int64

In [8]:
data_df = data_df[data_df.type != 'manufactured']
data_df = data_df[data_df.type != 'in-law']
data_df = data_df[data_df.type != 'land']
data_df = data_df[data_df.type != 'assisted living']

In [9]:
data_df.type.value_counts()

apartment        318032
house             33266
townhouse         15885
condo              6238
duplex             5047
cottage/cabin       861
loft                693
flat                531
Name: type, dtype: int64

In [10]:
data_df['type'] = data_df['type'].replace(['condo','loft','flat',],'apartment')
data_df['type'] = data_df['type'].replace(['townhouse','duplex','cottage/cabin'],'house')

In [11]:
data_df.type.value_counts()

apartment    325494
house         55059
Name: type, dtype: int64

In [12]:
def ishouse(row):
    if row['type'] == 'house':
        return int(1)
    else:
        return int(0)


data_df['Is_House']= data_df.apply(lambda k : ishouse(k),axis=1)

In [13]:
data_df.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state,Is_House
0,7049044568,https://reno.craigslist.org/apa/d/reno-beautif...,reno / tahoe,https://reno.craigslist.org,1148,apartment,1078,3,2.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/01616_daghmBUvTC...,Ridgeview by Vintage is where you will find al...,39.5483,-119.796,ca,0
1,7049047186,https://reno.craigslist.org/apa/d/reno-reduced...,reno / tahoe,https://reno.craigslist.org,1200,apartment,1001,2,2.0,0,...,0,0,w/d hookups,carport,https://images.craigslist.org/00V0V_5va0MkgO9q...,Conveniently located in the middle town of Ren...,39.5026,-119.789,ca,0
2,7043634882,https://reno.craigslist.org/apa/d/sparks-state...,reno / tahoe,https://reno.craigslist.org,1813,apartment,1683,2,2.0,1,...,0,0,w/d in unit,attached garage,https://images.craigslist.org/00t0t_erYqC6LgB8...,2BD | 2BA | 1683SQFTDiscover exceptional servi...,39.6269,-119.708,ca,0
3,7049045324,https://reno.craigslist.org/apa/d/reno-1x1-fir...,reno / tahoe,https://reno.craigslist.org,1095,apartment,708,1,1.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/00303_3HSJz75zlI...,MOVE IN SPECIAL FREE WASHER/DRYER WITH 6 OR 12...,39.4477,-119.771,ca,0
4,7049043759,https://reno.craigslist.org/apa/d/reno-no-long...,reno / tahoe,https://reno.craigslist.org,289,apartment,250,0,1.0,1,...,0,1,laundry on site,NaN,https://images.craigslist.org/01616_fALAWFV8zQ...,"Move In Today: Reno Low-Cost, Clean & Furnishe...",39.5357,-119.805,ca,0


In [14]:
data_df['weights'] = np.where(data_df['Is_House'] == 1, 0.15, 0.85)
sample_df = data_df.sample(frac=.2, random_state=111, weights='weights')

In [15]:
sample_df.dtypes

id                           int64
url                         object
region                      object
region_url                  object
price                        int64
type                        object
sqfeet                       int64
beds                         int64
baths                      float64
cats_allowed                 int64
dogs_allowed                 int64
smoking_allowed              int64
wheelchair_access            int64
electric_vehicle_charge      int64
comes_furnished              int64
laundry_options             object
parking_options             object
image_url                   object
description                 object
lat                        float64
long                       float64
state                       object
Is_House                     int64
weights                    float64
dtype: object

In [16]:
data_train, data_test = train_test_split(sample_df, test_size=0.2)

In [17]:
data_train.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state', 'Is_House', 'weights'],
      dtype='object')

In [18]:
lemmatizer = WordNetLemmatizer()
count_vectorizer = CountVectorizer(binary=True)
tfidf_vector = TfidfVectorizer(ngram_range=(1,3), stop_words='english')

In [19]:
# Initializing the lists for TRAINING DATA 
train_token_x = []

# Running the loop to tokenize, lemmatize, and remove the stop words from the reviews
for idx,row in data_train.iterrows():
    token_text = word_tokenize(str(row['description']).lower())
    lemmatized_text = [lemmatizer.lemmatize(token) for token in token_text if token.isalpha()]
    text_remove_stopwords = [token for token in lemmatized_text if not token in stopwords.words('english') if token.isalpha()]
    token1 = ''
    for token in text_remove_stopwords:
        token1 = token1 + ' ' + token
    train_token_x.append(token1)


tfidf_vector.fit(train_token_x)
train_tfidf_x = tfidf_vector.fit_transform(train_token_x)

train_y = data_train.Is_House

# Initializing the lists for TESTING DATA 
test_token_x = []

for idx,row in data_test.iterrows():
    token_text = word_tokenize(str(row['description']).lower())
    lemmatized_text = [lemmatizer.lemmatize(token) for token in token_text if token.isalpha()]
    text_remove_stopwords = [token for token in lemmatized_text if not token in stopwords.words('english') if token.isalpha()]
    token2 = ''
    for token in text_remove_stopwords:
        token2 = token2 + ' ' + token
    test_token_x.append(token2)


test_tfidf_x = tfidf_vector.transform(test_token_x)

test_y = data_test.Is_House



In [20]:
train_tfidf_x.shape, train_y.shape

((60888, 2097232), (60888,))

In [21]:
test_tfidf_x.shape, test_y.shape

((15223, 2097232), (15223,))

In [22]:
#Logistic Regression Model
logistic_regression = LogisticRegression(random_state=10)
logi_regr = logistic_regression.fit(train_tfidf_x, train_y)
logi_predicted_y = logi_regr.predict(test_tfidf_x)
print("The prediction accuracy for Logistic Regression is:", np.mean(logi_predicted_y == test_y)*100, "%")



# Random Forest
random_forest = RandomForestClassifier(n_estimators = 100, random_state=10)
model_rf = random_forest.fit(train_tfidf_x, train_y)
rf_predicted_y = model_rf.predict(test_tfidf_x)
print("The prediction accuracy for Random Forest is:", np.mean(rf_predicted_y == test_y)*100, "%")

The prediction accuracy for Logistic Regression is: 96.91256651120015 %
The prediction accuracy for Random Forest is: 97.01767062996781 %


In [23]:
#Predicting for actual data

act_data_df = pd.read_csv('/Users/harishgavva/Desktop/BAIM/Mod-4/Unstructured/Project-Data.csv')

# List for Actual Data 
act_token_x = []

for idx,row in act_data_df.iterrows():
    token_text = word_tokenize(str(row['Description']).lower())
    lemmatized_text = [lemmatizer.lemmatize(token) for token in token_text if token.isalpha()]
    text_remove_stopwords = [token for token in lemmatized_text if not token in stopwords.words('english') if token.isalpha()]
    token3 = ''
    for token in text_remove_stopwords:
        token3 = token3 + ' ' + token
    act_token_x.append(token3)

act_tfidf_x = tfidf_vector.transform(act_token_x)

predicted_y_act = model_rf.predict_proba(act_tfidf_x)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/harishgavva/Desktop/BAIM/Mod-4/Unstructured/Project-Data.csv'

In [ ]:
pred_y_act_df = pd.DataFrame(predicted_y_act,columns = ['is_house'])

In [ ]:
def housetype(row):
    if row['is_house'] == 1:
        return 'house'
    else:
        return 'apartment'
pred_y_act_df['type'] = pred_y_act_df.apply(lambda k :housetype(k),axis=1)

pred_y_act_df.is_house.value_counts()

In [ ]:

###Part 2. Topic Model

## 1. Transform those reviews into a term-document matrix, lemmatize all the words, remove the stop-words and
# punctuations, set the minimal document frequency for each term to be 5 and include uni-gram and bi-gram.
# Load the file
assignment_file = open('/Users/harishgavva/Desktop/BAIM/Mod-4/Unstructured/Project-Data.csv',encoding="utf-8")
assignment_reader = csv.reader(assignment_file)
review_data = list(assignment_reader)

review_list = []
for review in review_data:
    review_list.append(review[4])


# Tokenization and lemmatization
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t.isalpha()]


# Remove Stopwords and punctuation directly in Countvectorizer
my_stop_words = [t for t in (set(stopwords.words('english'))-set(['not','no','dont']))]

self_defined_stop_words = ['wa', 'quot', 'ha', 'gt', 'also','contact','contact info','home',
                           'community','info','show','show info','today','call','de','la','en','el','para',
                           'con','de la','un','available','â','â â','u','ri','http','hour','feature','offer',
                           'del','los','una','se','por','que','located','one','indianapolis','apartment','qr code',
                           'life','live','code','link','qr',"gene", "glick"] # These are extra stop words I defined after running the topics many times
my_stop_words.extend(self_defined_stop_words)
vectorizer = TfidfVectorizer(stop_words=my_stop_words, tokenizer=LemmaTokenizer(), encoding='utf-8', min_df=10,
                             ngram_range=(2, 3))
review_vect = vectorizer.fit_transform(review_list)

# 2. Use the LDA model to extract the topics of each document assuming there are 6 topics.
from sklearn.decomposition import LatentDirichletAllocation

terms = vectorizer.get_feature_names()
lda = LatentDirichletAllocation(n_components=6, random_state=0).fit(review_vect)
doc_topic = lda.transform(review_vect)  # 1000 by 6
term_document_matrix = doc_topic.transpose()  # 6 by 1000


# 4. Find the top-5 terms (terms with the top-5 highest weights) for each of the 6 topics.
# Based on those terms, describe what those topics are about.
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(", ".join([terms[i] for i in topic.argsort()[:-15 - 1:-1]]))


    

In [ ]:
Topic 0: Windsor park


Topic 1: Abney Lake


Topic 2: Airport


Topic 3: Busline


Topic 4: Prime Location


Topic 5: Lighthouse Landing


In [ ]:


project_data_df = pd.read_csv('/Users/harishgavva/Desktop/BAIM/Mod-4/Unstructured/Project-Data.csv')

topic_df = pd.DataFrame(doc_topic,columns=['Windsor Park','Abney Lake','Near Airport','Near Busline','Prime Location','Lighthouse Landing'])

topic_df['Tag'] = topic_df.idxmax(axis=1)


In [ ]:
final_df = pd.concat([project_data_df,topic_df['Tag']],axis=1)
final_df

In [ ]:
data_df.head()